# CTT Generator v1.0.0

Orrery allows for lots of flexibility in test creation, which makes presets rather wordy. 
For most project that flexibility is not needed. For example, in most tests all items have the same number of steps and each question has the same text and exactly one item. 

This utility scripts allows to generate Orrery's presets from much shorter templates. It is meant for manual use only and thus is not secure at all. It doesn't have any validation or error management. This is intentional though, because if generation is fails at any step, test template is not correct. Consistency checkes are handled by Orrery itself while loading tests from presets. 

In [1]:
import yaml
from datetime import datetime
from collections import Counter
from os import listdir
from os.path import isfile, join

In [2]:
GENERATOR = "CTT"
VERSION = "v1"
INPUT_PATH = f"../data/templates/{GENERATOR}-{VERSION}".lower()
OUTPUT_PATH = "../data/presets"
DRY = False # don't write files
ALL = False # regenerate all


class Dumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(Dumper, self).increase_indent(flow, False)

    
    
def make_header(template: str) -> str:
    res = [
        '# generated, try not to edit',
        f'# by: {GENERATOR}-{VERSION}',
        f'# at: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}',
        f'# template: {template}',
    ]
    return "\n".join(res)


def generate_presets(input_path: str, output_path: str, dry_run=True, regen_all=False):
    templates = [f for f in listdir(INPUT_PATH) if isfile(join(INPUT_PATH, f))]
    print("found", len(templates), "templates")
    for template in templates:
        regen = "yes"
        if not regen_all:
            regen = input(f"Processing {template}. Gerenerate? [yes|NO]")
            
        if regen != "yes":
            print("skipping...")
            continue
            
        publish = "yes"
        if not regen_all:
            publish = input(f"Processing {template}. Publish? [yes|NO]")

        with open(join(INPUT_PATH, template), 'r') as stream:
            test = yaml.safe_load(stream)
            
        test["publish"] = publish == "yes"
            
        body = make_preset(test)
        data = "\n".join([make_header(join(INPUT_PATH, template)), body])
        
        if not dry_run:
            filename = join(output_path, f'test.{test["code"]}.yaml')
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(data)
            print("saved to file:", filename)
        else:
            print("DRY RUN; generated following sweet yaml:\n", "-"*3, data, "-"*3, sep = '\n')
            
        print("processing template done:", template)
      
    
def scale_code(t, raw_scale_code):
    codes = list(t["scales"].keys())
    code = raw_scale_code.strip("-")
    number = codes.index(code) + 1
    return f'{t["code"]}-{number:02d}-{code}'


def make_display(t):
    return {
        "questionsPerPage": t["question"]["page"],
        "randomizeOrder": t["randomize"],
    }


def make_interpretations(t, scale):
    ranges = []
    if t["type"] == "sten":
        for i in range(t["intervals"]):
            ranges.append([9/(t["intervals"])*(i)+1, 9/(t["intervals"])*(i+1)+1])
    elif t["type"] == "perc":
        for i in range(t["intervals"]):
            ranges.append([100/(t["intervals"])*(i), 100/(t["intervals"])*(i+1)])
    else:
        print("INTERVALS GENERATION IS NOT DEFINED FOR TYPE", t["type"], "!!!")
        raise

    return [
        {
            "range": ranges[i],
            "translations": [
                {
                    "locale": loc,
                    "content": scale["interpretations"][i][loc]
                } for loc in t["locales"]
            ]
        } for i in range(len(ranges))
    ]
    

def make_scales(t):
    return [
        {
            "code": scale_code(t, code),
            "type": t["type"],
            "translations": [
                {
                    "locale": loc,
                    "title": s[loc][0],
                    "description": s[loc][1],
                    "abbreviation": s[loc][2],
                } for loc in t["locales"]
            ],
            "interpretations": make_interpretations(t, s)
        } for code, s in t["scales"].items()
    ]


def make_items(t, scales):
    item_codes = []
    item_map = { s["code"]:[] for s in scales}
    item_counter = Counter()
    for i, item in enumerate(t["items"]):
        sc = scale_code(t, item["scale"])
        item_counter.update([sc])
        item_code = f'{sc}-{item_counter[sc]:03d}'
        item_map[sc].append({
            "code": item_code,
            "steps": t["steps"],
            "reverse": item["scale"][0] == "-",
            "translations": [
                {
                    "locale": loc,
                    "content": item[loc],
                } for loc in t["locales"]
            ]
        })
        item_codes.append(item_code)
    return (item_map, item_codes)


def make_test_translations(t):
    return [
        {
            "locale": loc,
            "title": t[loc][0],
            "description": t[loc][1],
            "instruction": t[loc][2],
            "details": t[loc][3],
            "preambule": t[loc][4],
        } for loc in t["locales"]
    ]


def make_preset(t):
    scales = make_scales(t)

    item_map, item_codes = make_items(t, scales)   

    questions = [
        {
            "code": f'{t["code"]}-{i+1:03d}',
            "order": (i+1)*10,
            "type": "simple",
            "items": [{ "code":code for code in [c] }],
            "translations": [
                {
                    "locale": loc,
                    "content": t["question"][loc],
                } for loc in t["locales"]
            ],
        } for i, c in enumerate(item_codes)
    ]

    result = {
        "code": t["code"],
        "published": t["publish"],
        "availableLocales": t["locales"],
        "tags": t["tags"],
        "forceUpdate": True,
        "translations": make_test_translations(t),
        "scales": [{**s, "items": item_map[s["code"]]} for s in scales],
        "questions": questions,
        "display": make_display(t),
    }
    
    return yaml.dump(result, sort_keys=False, allow_unicode=True, Dumper=Dumper, default_flow_style=False)


In [8]:
%%time
generate_presets(INPUT_PATH, OUTPUT_PATH, DRY, ALL)

found 7 templates
Processing hexaco.yaml. Gerenerate? [yes|NO]
skipping...
Processing tipi.yaml. Gerenerate? [yes|NO]
skipping...
Processing riasec-personality-wisc.yaml. Gerenerate? [yes|NO]
skipping...
Processing political-compass.yaml. Gerenerate? [yes|NO]yes
Processing political-compass.yaml. Publish? [yes|NO]yes
saved to file: ../data/presets/test.political-compass.yaml
processing template done: political-compass.yaml
Processing riasec-combined-wisc.yaml. Gerenerate? [yes|NO]
skipping...
Processing riasec-activity-wisc.yaml. Gerenerate? [yes|NO]
skipping...
Processing fipi.yaml. Gerenerate? [yes|NO]
skipping...
CPU times: user 145 ms, sys: 14.1 ms, total: 159 ms
Wall time: 8.22 s
